In [1]:
import os
import sys
import importlib
import math
import time

import itertools
from collections import defaultdict

import numpy as np
import pandas as pd
import scipy.stats as ss
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.auto import tqdm as tqdm_auto
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import pytorch_lightning as pl

from IPython.display import clear_output

In [2]:
sys.path.append("../model/")
import utrdata_cl as utrdata
from legnet import LegNetClassifier
from pl_regressor import RNARegressor

## Loading data

In [3]:
PATH_FROM = "./UTR3_zscores_replicateagg.csv"
df = pd.read_csv(PATH_FROM)

In [5]:
num_classes = df["cell_type"].unique().shape[0]
num_classes

5

In [6]:
splits = dict(tuple(df.groupby('fold')))
for split_df in splits.values():
    split_df.reset_index(drop=True, inplace=True)
splits["val"].head()

,seq,cell_type,fold,1,2,3,4,mass_center,mass_center_mean,diff,zscore,mass_center_std
0,AAACAAATGTAGCAGGCACAGCGTGGGGTGGACAGTCAGCTGTCGG...,c1,val,68.449816,60.870873,47.590087,41.838243,2.287164,2.29667,-0.009506,-0.062351,0.152462
1,AAACAAATGTAGCAGGCACAGCGTGGGGTGGACAGTCAGCTGTCGG...,c17,val,72.812510,53.335530,34.751567,42.078913,2.227102,2.29667,-0.069567,-0.456294,0.152462
2,AAACAAATGTAGCAGGCACAGCGTGGGGTGGACAGTCAGCTGTCGG...,c2,val,71.853249,67.896625,57.267869,55.146572,2.379545,2.29667,0.082875,0.543581,0.152462
3,AAACAAATGTAGCAGGCACAGCGTGGGGTGGACAGTCAGCTGTCGG...,c4,val,77.312032,66.148593,64.039833,77.299136,2.496230,2.29667,0.199560,1.308917,0.152462
4,AAACAAATGTAGCAGGCACAGCGTGGGGTGGACAGTCAGCTGTCGG...,c6,val,86.538512,56.970286,42.241073,32.322653,2.093308,2.29667,-0.203362,-1.333853,0.152462


In [7]:
batch_size = 1024
steps_per_epoch = max(1, splits["train"].shape[0] // batch_size)

# batch_per_epoch = 128  # None

# if batch_per_epoch is None:
#     # batch_per_epoch = int(np.ceil(splits["train"].shape[0] / batch_size))
#     batch_per_epoch = splits["train"].shape[0] // batch_size  # drop_last=True @ DataLoader

epochs = 20

In [8]:
num_workers = 32

In [9]:
def launch_model(
    seed: int,
    train_ds_kws: dict,
    val_ds_kws: dict,
    model_class,
    model_kws: dict,
    criterion_class,
    criterion_kws: dict,
    optimizer_class,
    optimizer_kws: dict,
    lr_scheduler_class,
    lr_scheduler_kws: dict,
    test_time_validation: bool,
    epochs: int = epochs,
):
    pl.seed_everything(seed)

    # Creating Datasets
    train_set = utrdata.UTRData(
        df=splits["train"],
        **train_ds_kws,
    )
    val_set = utrdata.UTRData(
        df=splits["val"],
        **val_ds_kws,
    )

    assert train_set.num_channels == val_set.num_channels
    try:
        div_factor = val_ds_kws["augment_kws"]["shift_left"] + \
                     val_ds_kws["augment_kws"]["shift_right"] + 1
    except KeyError:
        div_factor = 1

    # Creating DataLoaders
    dl_train = DataLoader(
        train_set,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=True,
        drop_last=True
    )
    # dl_train = utrdata.DataLoaderWrapper(dl_train, batch_per_epoch=batch_per_epoch)
    dl_val = DataLoader(
        val_set,
        batch_size=batch_size // div_factor,
        num_workers=num_workers,
        shuffle=False,
        drop_last=False
    )

    model = RNARegressor(
        model_class=model_class,
        model_kws=model_kws | dict(
            in_channels=train_set.num_channels
        ),
        criterion_class=criterion_class,
        criterion_kws=criterion_kws,
        optimizer_class=optimizer_class,
        optimizer_kws=optimizer_kws,
        lr_scheduler_class=lr_scheduler_class,
        lr_scheduler_kws=lr_scheduler_kws,
        test_time_validation=test_time_validation,
    )
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        dirpath="saved_models",
        save_top_k=1,
        save_last=False,
        monitor="val_pearson_r_0",
        mode="max"
    )
    progressbar_callback = pl.callbacks.TQDMProgressBar(refresh_rate=0.5)

    logger = pl.loggers.tensorboard.TensorBoardLogger("tb_logs", name=model.model_name)
    trainer = pl.Trainer(
        callbacks=[checkpoint_callback, progressbar_callback],
        logger=logger,
        accelerator="gpu",
        devices=1,
        deterministic=True,
        max_epochs=epochs,
        num_sanity_val_steps=0,
        # gradient_clip_val=1e-5,
        # gradient_clip_algorithm="norm",
    )
    trainer.fit(model=model, train_dataloaders=dl_train, val_dataloaders=dl_val)
    best_model = RNARegressor.load_from_checkpoint(checkpoint_callback.best_model_path)

    prediction = trainer.predict(model=best_model, dataloaders=dl_val)
    val_pred, val_real = zip(*prediction)
    val_pred = torch.concat(val_pred).numpy()
    val_real = torch.concat(val_real).numpy()
    val_df = splits["val"].copy()
    val_df["real_0"] = val_real[:, 0]
    val_df["real_1"] = val_real[:, 1]
    val_df["pred_0"] = val_pred[:, 0]
    val_df["pred_1"] = val_pred[:, 1]

    return trainer, val_df

In [10]:
checked = {
    "seed": [3],  # 3, 7, 13, 42, 100
    "features": [
        # ("sequence", "revcomp", "intensity", "positional", "conditions"),
        ("sequence", "positional", "conditions"),
    ],
    "augment_dict": [
        dict(
            extend_left=0,
            extend_right=0,
            shift_left=0,
            shift_right=0,
            revcomp=False,
        ),
    ],
    "epochs": [10],
    # "task_loss": [
    #     ("soft_classification", nn.CrossEntropyLoss),
    #     ("soft_regression", nn.MSELoss),
    #     ("regression", nn.MSELoss),
    # ]
}

for subset in itertools.product(
    *checked.values()
):
    PARAMS = dict(zip(checked.keys(), subset))
    AUGMENT_KEY = any(PARAMS["augment_dict"].values())
    AUGMENT_TEST_TIME = AUGMENT_KEY

    trainer_last, prediction_best_last = launch_model(
        seed=PARAMS["seed"],
        train_ds_kws=dict(
            construct_type="utr3",
            features=PARAMS["features"],  # ("sequence", "conditions", "positional", "revcomp")
            augment=AUGMENT_KEY,
            augment_test_time=False,
            augment_kws=PARAMS["augment_dict"],
        ),
        val_ds_kws=dict(
            construct_type="utr3",
            features=PARAMS["features"],  # ("sequence", "conditions", "positional", "revcomp")
            augment=False,
            augment_test_time=AUGMENT_TEST_TIME,
            augment_kws=PARAMS["augment_dict"],
        ),
        model_class=LegNetClassifier,
        model_kws=dict(
            seqsize=50,
            in_channels=4,  # IS REPLACED
            ks=3,
            out_channels=2,
            conv_sizes=(128, 64, 64, 32, 32),
            mapper_size=256,
            linear_sizes=None,
            use_max_pooling=False,
            final_activation=nn.Identity
        ),
        criterion_class=nn.MSELoss,
        criterion_kws=dict(),
        optimizer_class=torch.optim.AdamW,
        optimizer_kws=dict(
            # lr=0.01,
            weight_decay=0.1,
        ),
        lr_scheduler_class=torch.optim.lr_scheduler.OneCycleLR,
        lr_scheduler_kws=dict(
            max_lr=0.01,
            steps_per_epoch=steps_per_epoch,
            epochs=PARAMS["epochs"],
            pct_start=0.3,
            three_phase=False,
            cycle_momentum=True,
        ),
        test_time_validation=AUGMENT_TEST_TIME,
        epochs=PARAMS["epochs"],
    )

Global seed set to 3
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: tb_logs/LegNetClassifier_C128-64-64-32-32_M256_L1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | LegNetClassifier | 268 K 
1 | criterion | MSELoss          | 0     
-----------------------------------------------
268 K     Trainable params
0         Non-trainable params
268 K     Total params
1.073     Total estimated model params siz

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: 0it [00:00, ?it/s]

## Launching model

In [ ]:
plt.figure(figsize=(12, 8))

rows = 2
cols = 3

x = "mass_center"
y = "pred_1"

ax = plt.subplot(rows, cols, 1)
r = ss.pearsonr(prediction_best_last[x], prediction_best_last[y]).statistic
plt.scatter(prediction_best_last[x], prediction_best_last[y], color="C4", marker=".", alpha=0.05)
plt.text(0.1, 0.9, f"r = {r:.04f}", color="C3", transform=ax.transAxes)
plt.title("All cell lines")
plt.ylabel("Predicted bin mass center")

for i, (ct, ctdf) in enumerate(prediction_best_last.groupby(by="cell_type"), 2):
    r = ss.pearsonr(ctdf[x], ctdf[y]).statistic
    ax = plt.subplot(rows, cols, i)
    plt.scatter(ctdf[x], ctdf[y], color="C0", marker=".", alpha=0.1)
    plt.text(0.1, 0.9, f"$r_1$ = {r:.04f}", transform=ax.transAxes)
    plt.title(f"Cell line: {ct}")
    if (i - 1) // cols == rows - 1:
        plt.xlabel("Real bin mass center")
    if i % cols == 1:
        plt.ylabel("Predicted bin mass center")
plt.tight_layout()

plt.suptitle(PATH_FROM)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))

rows = 2
cols = 3

x = "real_0"
y = "pred_0"

ax = plt.subplot(rows, cols, 1)
r = ss.pearsonr(prediction_best_last[x], prediction_best_last[y]).statistic
plt.scatter(prediction_best_last[x], prediction_best_last[y], color="C4", marker=".", alpha=0.05)
plt.text(0.1, 0.9, f"r = {r:.04f}", color="C3", transform=ax.transAxes)
plt.title("All cell lines")
# plt.ylabel("Predicted bin mass center")

for i, (ct, ctdf) in enumerate(prediction_best_last.groupby(by="cell_type"), 2):
    r = ss.pearsonr(ctdf[x], ctdf[y]).statistic
    ax = plt.subplot(rows, cols, i)
    sns.scatterplot(x=ctdf[x], y=ctdf[y], color="C0", marker=".", alpha=0.5)
    plt.text(0.1, 0.85, f"$r_1$ = {r:.04f}", transform=ax.transAxes)
    plt.title(f"Cell line: {ct}")
    # if (i - 1) // cols == rows - 1:
    #     plt.xlabel("Real bin mass center")
    # if i % cols == 1:
    #     plt.ylabel("Predicted bin mass center")
plt.tight_layout()

plt.suptitle(PATH_FROM)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))

rows = 2
cols = 3

x = "real_0"
y = "pred_0"

ax = plt.subplot(rows, cols, 1)
r = ss.pearsonr(prediction_best_last[x], prediction_best_last[y]).statistic
plt.scatter(prediction_best_last[x], prediction_best_last[y], color="C4", marker=".", alpha=0.05)
plt.text(0.1, 0.9, f"r = {r:.04f}", color="C3", transform=ax.transAxes)
plt.title("All cell lines")
# plt.ylabel("Predicted bin mass center")

for i, (ct, ctdf) in enumerate(prediction_best_last.groupby(by="cell_type"), 2):
    xx = ctdf.groupby("seq")[x].mean()
    yy = ctdf.groupby("seq")[y].mean()
    r = ss.pearsonr(xx, yy).statistic
    ax = plt.subplot(rows, cols, i)
    plt.scatter(x=xx, y=yy, color="C0", marker=".", alpha=0.25)
    plt.text(0.1, 0.85, f"$r_1$ = {r:.04f}", transform=ax.transAxes)
    plt.title(f"Cell line: {ct}")
    # if (i - 1) // cols == rows - 1:
    #     plt.xlabel("Real bin mass center")
    # if i % cols == 1:
    #     plt.ylabel("Predicted bin mass center")
plt.tight_layout()

plt.suptitle(PATH_FROM)
plt.tight_layout()
plt.show()

---